# JAY BAKSHI : UB ID - 50206954

Installing and importing relevant packages and libraries.

In [ ]:
install.packages('dpylr', repos = 'https://cran.r-project.org')

In [ ]:
install.packages('DBI', repos="https://cran.r-project.org")

In [ ]:
install.packages('RSQLite', repos = 'http://cran.r-project.org')

In [ ]:
install.packages('sqldf', repos="https://cran.r-project.org")

In [ ]:
library(dplyr)

In [ ]:
library(DBI)

In [ ]:
library(RSQLite)

In [ ]:
library(sqldf)

### Downloaded the dataset at https://www.kaggle.com/hugomathien/soccer

Accessing the database.sqlite using dplyr methods -

In [ ]:
my_db <- src_sqlite("my_db.sqlite", create=T)

In [ ]:
class(my_db)

In [ ]:
src_tbls(my_db)

Saving the Table Player and Team in their respective dataframes to be useful later

In [ ]:
playerdf <- read.csv(file="player.csv")

In [ ]:
playertbl <- tbl_df(playerdf)

In [ ]:
class(playertbl)

In [ ]:
p <- copy_to(my_db, playertbl)

In [ ]:
dim(playertbl)
head(playertbl, 10)

<b> Query - Which are the top 10 players by overall rating? </b>
<br/>Columns - 
<br/>playertbl - player_name, player_api_id, player_api_id, overall_rating

In [ ]:
q1 <- tbl(my_db, sql("SELECT DISTINCT player_api_id, player_name
                    FROM playertbl
                    ORDER BY overall_rating DESC
                    LIMIT 10"))

In [ ]:
q1

<b>Query 2 - Which are the top 10 goalkeepers by sum of gk attributes?</b>
<br/>Columns -
<br/>Player - player_name, player_api_id
<br/>Player_Attributes - player_api_id, gk_diving, gk_handling, gk_kicking, gk_positioning, gk_reflexes

In [ ]:
q2 <- tbl(my_db, sql("SELECT DISTINCT player_name
                            FROM playertbl
                            ORDER BY gk_all DESC
                            LIMIT 10"))

In [ ]:
q2

In [ ]:
teamdf <- read.csv(file="team.csv")
#class(teamdf)
#dim(teamdf)
#head(teamdf, 3)

In [ ]:
teamtbl <- tbl_df(teamdf)
#class(teamtbl)
#dim(teamtbl)
#head(teamtbl, 3)

<b>Which are the top 10 players by number of appearances (with any team they played with)?</b>
The plan is to extract the data about the player_id across the table spread in multiple columns to bring it all together, match it and then group by the same player_id so we can easily get a count of the appearances made by the player.

The field league_id has been added here to cater to the next question. Since, this splitting of data does take some time so I combined the effort to reduce unnecessary redundancy.

In [ ]:
p <- copy_to(my_db, teamtbl)

In [ ]:
q3_h1 <- sqldf("SELECT home_player_1 AS id, league_id AS lid FROM teamtbl")
q3_h2 <- sqldf("SELECT home_player_2 AS id, league_id AS lid FROM teamtbl")
q3_h3 <- sqldf("SELECT home_player_3 AS id, league_id AS lid FROM teamtbl")
q3_h4 <- sqldf("SELECT home_player_4 AS id, league_id AS lid FROM teamtbl")
q3_h5 <- sqldf("SELECT home_player_5 AS id, league_id AS lid FROM teamtbl")
q3_h6 <- sqldf("SELECT home_player_6 AS id, league_id AS lid FROM teamtbl")
q3_h7 <- sqldf("SELECT home_player_7 AS id, league_id AS lid FROM teamtbl")
q3_h8 <- sqldf("SELECT home_player_8 AS id, league_id AS lid FROM teamtbl")
q3_h9 <- sqldf("SELECT home_player_9 AS id, league_id AS lid FROM teamtbl")
q3_h10 <- sqldf("SELECT home_player_10 AS id, league_id AS lid FROM teamtbl")
q3_h11 <- sqldf("SELECT home_player_11 AS id, league_id AS lid FROM teamtbl")

q3_a1 <- sqldf("SELECT away_player_1 AS id, league_id AS lid FROM teamtbl")
q3_a2 <- sqldf("SELECT away_player_2 AS id, league_id AS lid FROM teamtbl")
q3_a3 <- sqldf("SELECT away_player_3 AS id, league_id AS lid FROM teamtbl")
q3_a4 <- sqldf("SELECT away_player_4 AS id, league_id AS lid FROM teamtbl")
q3_a5 <- sqldf("SELECT away_player_5 AS id, league_id AS lid FROM teamtbl")
q3_a6 <- sqldf("SELECT away_player_6 AS id, league_id AS lid FROM teamtbl")
q3_a7 <- sqldf("SELECT away_player_7 AS id, league_id AS lid FROM teamtbl")
q3_a8 <- sqldf("SELECT away_player_8 AS id, league_id AS lid FROM teamtbl")
q3_a9 <- sqldf("SELECT away_player_9 AS id, league_id AS lid FROM teamtbl")
q3_a10 <- sqldf("SELECT away_player_10 AS id, league_id AS lid FROM teamtbl")
q3_a11 <- sqldf("SELECT away_player_11 AS id, league_id AS lid FROM teamtbl")

Combining all the relevant records.

In [ ]:
h3_temp <- rbind(
q3_h1,
q3_h2,
q3_h3,
q3_h4,
q3_h5,
q3_h6,
q3_h7,
q3_h8,
q3_h9,
q3_h10,
q3_h11,
q3_a1,
q3_a2,
q3_a3,
q3_a4,
q3_a5,
q3_a6,
q3_a7,
q3_a8,
q3_a9,
q3_a10,
q3_a11)

In [ ]:
class(h3_temp)
head(h3_temp, 3)

In [ ]:
q3 <- sqldf("SELECT DISTINCT player_name, h3_temp.id, COUNT(*)
                FROM h3_temp, playertbl
                WHERE playertbl.player_api_id = h3_temp.id
                GROUP BY id")

In [ ]:
q3

<b>Which are the top 10 players by number of leagues they played in?</b>
<br/>Using the league_id with alias lid extracted from the data in the query made above, this was done to reduce the data recall time across the 2 tasks.

In [ ]:
q4 <- tbl(my_db, sql("SELECT DISTINCT player_name, COUNT(*)
                        FROM h3_temp, playertbl
                        WHERE playertbl.player_api_id = h3_temp.id
                        GROUP BY lid"))

In [ ]:
q4

<b>Which are the top 10 teams by sum of build up play attributes?</b>
<br/> I have preadded the columns dealing with the data of buildUpPlay, because we were interested in the total of them and not individual and also they help save a space by reducing relevant number of columns x records.

In [ ]:
q5 <- tbl(my_db, sql("SELECT DISTINCT team_long_name, buildUpPlay
                        FROM teamtbl
                        ORDER BY buildUpPlay DESC
                        LIMIT 10"))

In [ ]:
q5

<b>Which are the top 10 teams by sum of chance creation attributes?</b>
<br/>Again summed up the columns of chanceCreation so as to avoid getting too many records in the teamtbl and also retain only the data that we are interested in for this assignment - the sum of those values.

In [ ]:
q6 <- tbl(my_db, sql("SELECT DISTINCT team_long_name, chanceCreation
                            FROM teamtbl
                            ORDER BY chanceCreation DESC
                            LIMIT 10"))

In [ ]:
q6

<b>Which are the top 10 teams by sum of defense attributes?</b>
<br/>Summed up defence values for the reason as above. I practically reduced my team.csv size from the tune of 3+ GB to ~160 MB with such merges and summation across columnar records.

In [ ]:
q7 <- tbl(my_db, sql("SELECT DISTINCT team_long_name, defence
                        FROM teamtbl
                        ORDER BY defence DESC
                        LIMIT 10"))

In [ ]:
q7

<b>Which are the top 10 teams by number of scored goals?</b>
<br/> Picked up this question out of the 3 options to choose from. The rationale is to have 2 subsets of goals data one for home_team and away_team each, so matching can be done better. Eventually group it by the same team id to arrive at a combined set of goals scored by the team.

In [ ]:
colnames(teamtbl)

In [ ]:
hgoal <- data.frame(tbl(my_db, sql("SELECT DISTINCT home_team_api_id AS id, home_team_goal AS goal, match_api_id AS mid
                                    FROM teamtbl")))

In [ ]:
hgoaltbl <- copy_to(my_db, tbl_df(hgoal))

In [ ]:
class(hgoal)

In [ ]:
agoal <- data.frame(tbl(my_db, sql("SELECT DISTINCT away_team_api_id AS id, away_team_goal AS goal, match_api_id AS mid
                                    FROM teamtbl")))

In [ ]:
agoaltbl <- copy_to(my_db, tbl_df(agoal))

In [ ]:
totgoal <- data.frame(tbl(my_db, sql("SELECT DISTINCT hgoal.id AS teamid, (hgoal.goal+agoal.goal) AS teamgoal
                                        FROM hgoal, agoal
                                        WHERE hgoal.id = agoal.id
                                        ORDER BY teamgoal DESC
                                        LIMIT 10")))

In [ ]:
totgoal